In [62]:
import math
from seal import *
from seal_helper import *

In [71]:
    print_example_banner("Example: CKKS Basics")

    parms = EncryptionParameters(scheme_type.CKKS)

    poly_modulus_degree = 8192
    parms.set_poly_modulus_degree(poly_modulus_degree)
    parms.set_coeff_modulus(CoeffModulus.Create(
        poly_modulus_degree, [50, 30, 30, 30, 30, 40]))

    scale = pow(2.0, 30)
    context = SEALContext.Create(parms)
    print_parameters(context)

    keygen = KeyGenerator(context)
    public_key = keygen.public_key()
    secret_key = keygen.secret_key()
    relin_keys = keygen.relin_keys()

    encryptor = Encryptor(context, public_key)
    evaluator = Evaluator(context)
    decryptor = Decryptor(context, secret_key)

    encoder = CKKSEncoder(context)
    slot_count = encoder.slot_count()
    print("Number of slots: " + str(slot_count))

+--------------------------------------+
|         Example: CKKS Basics         |
+--------------------------------------+
/
| Encryption parameters:
| scheme: CKKS
| poly_modulus_degree: 8192
| coeff_modulus size: 210(50 + 30 + 30 + 30 + 30 + 40) bits
\
Number of slots: 4096


In [64]:
    A = DoubleVector([100])
    B = DoubleVector([1e-3])

    print("Input vector: ")
    print_vector(A, 1, 7)
    print_vector(B, 1, 7)

    print("Evaluating polynomial 5*a^5*b^4")

    plain_02 = Plaintext()  # plain_02 = after encoding 0.2, 
                            # plain_0002 = after encoding 0.002
    plain_5 = Plaintext()
    encoder.encode(0.2, scale, plain_02)
    encoder.encode(5, scale, plain_5)
    
    plain_a = Plaintext()
    plain_b = Plaintext()
    print("-" * 50)
    print("Encode input vectors.")
    encoder.encode(A, scale, plain_a)
    encoder.encode(B, scale, plain_b)

    cipher_a = Ciphertext()
    cipher_b = Ciphertext() 
    encryptor.encrypt(plain_a, cipher_a)
    encryptor.encrypt(plain_b, cipher_b)



Input vector: 

    [ 100.0000000 ]


    [ 0.0010000 ]

Evaluating polynomial 5*a^5*b^4
--------------------------------------------------
Encode input vectors.


In [65]:
    print_example_banner("level 3")

# 5*a computation    
    
    print("-" * 50)
    print("Compute and rescale a*5.")
    cipher_5_a = Ciphertext()  # 5_a : 5 * a
                               # a2 : a^2
                               # 5_a5_b4 : 5 * a^5 * b^4
    evaluator.multiply_plain(cipher_a, plain_5, cipher_5_a)
    print("    + Scale of a*5 before rescale: " +
          "%.0f" % math.log(cipher_5_a.scale(), 2) + " bits")
   
    #evaluator.relinearize_inplace(cipher_5_a, relin_keys)

    evaluator.rescale_to_next_inplace(cipher_5_a)
    print("    + Scale of a*5 after rescale: " +
          "%.0f" % math.log(cipher_5_a.scale(), 2) + " bits") 
    
# a*b computation  

    print("-" * 50)
    print("Compute, relinearize, and rescale a*b.")
    cipher_a_b = Ciphertext()
    evaluator.multiply(cipher_a, cipher_b, cipher_a_b)
    evaluator.relinearize_inplace(cipher_a_b, relin_keys)
    print("    + Scale of a*b before rescale: " +
          "%.0f" % math.log(cipher_a_b.scale(), 2) + " bits")
    evaluator.rescale_to_next_inplace(cipher_a_b)
    print("    + Scale of a*b after rescale: " +
          "%.0f" % math.log(cipher_a_b.scale(), 2) + " bits")    

+-------------------------+
|         level 3         |
+-------------------------+
--------------------------------------------------
Compute and rescale a*5.
    + Scale of a*5 before rescale: 60 bits
    + Scale of a*5 after rescale: 30 bits
--------------------------------------------------
Compute, relinearize, and rescale a*b.
    + Scale of a*b before rescale: 60 bits
    + Scale of a*b after rescale: 30 bits


In [66]:
    print_example_banner("level 2")

# a^2*b^2 computation  

    print("-" * 50)
    print("Compute, relinearize, and rescale a^2*b^2.")  
    cipher_a2_b2 = Ciphertext()
    evaluator.square(cipher_a_b, cipher_a2_b2)
    evaluator.relinearize_inplace(cipher_a2_b2, relin_keys)
    print("    + Scale of a^2*b^2 before rescale: " +
          "%.0f" % math.log(cipher_a2_b2.scale(), 2) + " bits")
    evaluator.rescale_to_next_inplace(cipher_a2_b2)
    print("    + Scale of a^2*b^2 after rescale: " +
          "%.0f" % math.log(cipher_a2_b2.scale(), 2) + " bits")    
    

+-------------------------+
|         level 2         |
+-------------------------+
--------------------------------------------------
Compute, relinearize, and rescale a^2*b^2.
    + Scale of a^2*b^2 before rescale: 60 bits
    + Scale of a^2*b^2 after rescale: 30 bits


In [67]:
    print_example_banner("level 1")

    
# a^4*b^4 computation  

    print("-" * 50)
    print("Compute, relinearize, and rescale a^4*b^4.")  
    cipher_a4_b4 = Ciphertext()
    evaluator.square(cipher_a2_b2, cipher_a4_b4)
    evaluator.relinearize_inplace(cipher_a4_b4, relin_keys)
    print("    + Scale of a^4*b^4 before rescale: " +
          "%.0f" % math.log(cipher_a4_b4.scale(), 2) + " bits")
    evaluator.rescale_to_next_inplace(cipher_a4_b4)
    print("    + Scale of a^4*b^4 after rescale: " +
          "%.0f" % math.log(cipher_a4_b4.scale(), 2) + " bits")

+-------------------------+
|         level 1         |
+-------------------------+
--------------------------------------------------
Compute, relinearize, and rescale a^4*b^4.
    + Scale of a^4*b^4 before rescale: 60 bits
    + Scale of a^4*b^4 after rescale: 30 bits


In [68]:
    print_example_banner("level 0")

# mod switch
    print("-" * 50)
    print("Normalize encryption parameters to the lowest level.")
    last_parms_id = cipher_a4_b4.parms_id()
    evaluator.mod_switch_to_inplace(cipher_5_a, last_parms_id)
    
# 5*a^5*b^4 computation  

    print("-" * 50)
    print("Compute, relinearize, and rescale (a^4*b^4)*(5*a)=5*a^5*b^4.")
    cipher_5_a5_b4 = Ciphertext()
    evaluator.multiply(cipher_a4_b4, cipher_5_a, cipher_5_a5_b4)
    evaluator.relinearize_inplace(cipher_5_a5_b4, relin_keys)
    print("    + Scale of 5*a^5*b^4 before rescale: " +
          "%.0f" % math.log(cipher_5_a5_b4.scale(), 2) + " bits")
    evaluator.rescale_to_next_inplace(cipher_5_a5_b4)
    print("    + Scale of 5*a^5*b^4 after rescale: " +
          "%.0f" % math.log(cipher_5_a5_b4.scale(), 2) + " bits")

+-------------------------+
|         level 0         |
+-------------------------+
--------------------------------------------------
Normalize encryption parameters to the lowest level.
--------------------------------------------------
Compute, relinearize, and rescale (a^4*b^4)*(5*a)=5*a^5*b^4.
    + Scale of 5*a^5*b^4 before rescale: 60 bits
    + Scale of 5*a^5*b^4 after rescale: 30 bits


In [69]:
    print_example_banner("computation result")

    # Print the true result.
    
    plain_temp = Plaintext()
    print("-" * 50)
    print("Decrypt and decode (1*x)^2.")
    print("    + Expected result:")
    true_result = [5*100**5*0.001**4]
    #for x in A:
        #true_result.append(5*100**5*0.001**4)
    print_vector(true_result, 3, 7)

    
    
    # Decrypt, decode, and print the result.
    
    decryptor.decrypt(cipher_5_a5_b4, plain_temp)
    temp = DoubleVector()
    encoder.decode(plain_temp, temp)
    print("    + Computed result ...... Correct.")
    print_vector(temp, 1, 7)
    

+------------------------------------+
|         computation result         |
+------------------------------------+
--------------------------------------------------
Decrypt and decode (1*x)^2.
    + Expected result:

    [ 0.0500000 ]

    + Computed result ...... Correct.

    [ 0.0501760, ..., -0.0000010 ]

